<a href="https://colab.research.google.com/github/SuyeonSim/Data-analystic-Project/blob/main/03_AOV_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# 저장했던 중간 데이터 불러오기
df_product = pd.read_csv('modori_preprocessed_items.csv')
df_aov = pd.read_csv('modori_preprocessed_orders.csv')

# 날짜 컬럼은 불러오면 다시 문자열이 되므로, 분석 전에 다시 datetime으로 변환해줍니다.
df_aov['주문일자'] = pd.to_datetime(df_aov['주문일자'])

print("🚀 분석용 데이터 로드 완료! 객단가 분석을 시작합니다.")

🚀 분석용 데이터 로드 완료! 객단가 분석을 시작합니다.


### **최근 1년 객단가 분석**

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 1. 날짜 데이터 형식 변환 및 기준일 설정
df_aov['주문일자'] = pd.to_datetime(df_aov['주문일자'])
analysis_end_date = df_aov['주문일자'].max() # 데이터상의 마지막 주문일 기준
analysis_start_date = analysis_end_date - pd.DateOffset(years=1)

# 2. 최근 1년 데이터 필터링
df_aov_1y = df_aov[df_aov['주문일자'] >= analysis_start_date].copy()

# 3. 금액 구간(Bins) 및 라벨 설정
max_val = df_aov_1y['총 결제금액'].max()
bins = [0, 10000, 30000, 70000, 100000, 130000, 160000, 200000, 300000, 400000, max_val + 1]
labels = ['0-1만원', '1-3만원', '3-7만원', '7-10만원', '10-13만원', '13-16만원', '16-20만원', '20-30만원', '30-40만원', '40만원 이상']

# 4. 데이터 구간화 (Binning)
df_aov_1y['금액구간'] = pd.cut(df_aov_1y['총 결제금액'], bins=bins, labels=labels, right=False)

# 5. 구간별 통계 집계
segment_1y = df_aov_1y.groupby('금액구간', observed=True).agg(
    주문건수=('주문번호', 'count'),
    총매출액=('총 결제금액', 'sum'),
    평균결제액=('총 결제금액', 'mean')
).reset_index()

# 6. 비중 및 누적비중 계산
total_count = segment_1y['주문건수'].sum()
segment_1y['비중(%)'] = (segment_1y['주문건수'] / total_count * 100).round(2)
segment_1y['누적비중(%)'] = segment_1y['비중(%)'].cumsum().round(2)

# 7. 결과 출력
print(f"📅 분석 기간: {analysis_start_date.date()} ~ {analysis_end_date.date()}")
print(f"📊 최근 1년 주문 데이터 기준 전략적 금액 구간 분석")
display(segment_1y)

📅 분석 기간: 2025-01-21 ~ 2026-01-21
📊 최근 1년 주문 데이터 기준 전략적 금액 구간 분석


,금액구간,주문건수,총매출액,평균결제액,비중(%),누적비중(%)
0,1-3만원,1,23911,23911.0,7.69,7.69
1,3-7만원,2,133727,66863.5,15.38,23.07
2,7-10만원,1,74247,74247.0,7.69,30.76
3,10-13만원,1,122878,122878.0,7.69,38.45
4,13-16만원,3,420102,140034.0,23.08,61.53
5,20-30만원,3,768207,256069.0,23.08,84.61
6,30-40만원,1,370166,370166.0,7.69,92.30
7,40만원 이상,1,593892,593892.0,7.69,99.99


In [6]:
import pandas as pd

# 1. 금액 구간 정보를 전체 상품 데이터에 매핑
# 주문번호를 기준으로 df_aov_1y의 '금액구간' 컬럼을 df_product와 병합합니다.
df_product_segment = pd.merge(
    df_product,
    df_aov_1y[['주문번호', '금액구간']],
    on='주문번호',
    how='inner'
)

# 2. 금액 구간별 상품 판매 빈도 집계
# 구간별로 어떤 상품이 가장 많이 팔렸는지 카운트합니다.
segment_product_stats = df_product_segment.groupby(['금액구간', '주문상품명'], observed=True).size().reset_index(name='판매건수')

# 3. 구간별 판매량 기준 내림차순 정렬 및 상위 5개 추출
segment_product_stats = segment_product_stats.sort_values(['금액구간', '판매건수'], ascending=[True, False])
top_products_by_segment = segment_product_stats.groupby('금액구간').head(5)

# 4. 결과 출력
print("📊 금액 구간별 인기 구매 상품 Top 5 (최근 1년)")
for segment in top_products_by_segment['금액구간'].unique():
    print(f"\n[{segment} 구간]")
    display(top_products_by_segment[top_products_by_segment['금액구간'] == segment].reset_index(drop=True))

📊 금액 구간별 인기 구매 상품 Top 5 (최근 1년)

[1-3만원 구간]


/tmp/ipython-input-287179766.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  top_products_by_segment = segment_product_stats.groupby('금액구간').head(5)


,금액구간,주문상품명,판매건수
0,1-3만원,무선 핸디 청소기,1



[3-7만원 구간]


,금액구간,주문상품명,판매건수
0,3-7만원,데일리 캔버스 토트백,1
1,3-7만원,미니멀리스트 식기세트,1
2,3-7만원,우드 그레인 조리도구,1
3,3-7만원,프리미엄 리넨 앞치마,1


...

,금액구간,주문상품명,판매건수
0,20-30만원,우드 그레인 조리도구,2
1,20-30만원,스마트 에어 써큘레이터,1
2,20-30만원,에코 프렌들리 가습기,1
3,20-30만원,프리미엄 리넨 앞치마,1



[30-40만원 구간]


,금액구간,주문상품명,판매건수
0,30-40만원,우드 그레인 조리도구,1



[40만원 이상 구간]


,금액구간,주문상품명,판매건수
0,40만원 이상,오가닉 코튼 타월세트,1
